In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
res_folders=os.listdir('../../results/')

In [2]:
model_folder='/home/mara/multitask_adversarial/results/NCOUNT_822/'
CONCEPT=['domain', 'ncount', 'narea']

In [3]:
import keras
keras.__version__
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [4]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = str(0)# str(hvd.local_rank())
keras.backend.set_session(tf.Session(config=config))
verbose=1 
init=tf.global_variables_initializer() #initialize_all_variables()
sess=tf.Session()
sess.run(init)

In [5]:
## Loading OS libraries to configure server preferences
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
import setproctitle
SERVER_NAME = 'ultrafast'
EXPERIMENT_TYPE='visualize_guided'
import time
import sys
import shutil
## Adding PROCESS_UC1 utilities
sys.path.append('../../lib/TASK_2_UC1/')
from models import *
from util import otsu_thresholding
from extract_xml import *
from functions import *                   
sys.path.append('../../lib/')
from mlta import *
import math
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = str(0)# str(hvd.local_rank())
keras.backend.set_session(tf.Session(config=config))

verbose=1 
"""loading dataset files"""
#rank = MPI.COMM_WORLD.rank
cam16 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/cam16_500/patches.h5py',  'r', libver='latest', swmr=True)
all500 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/all500/patches.h5py',  'r', libver='latest', swmr=True)
extra17 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/extra17/patches.h5py','r', libver='latest', swmr=True)
tumor_extra17=hd.File('/home/mara/adversarialMICCAI/data/ultrafast/1129-1155/patches.h5py', 'r', libver='latest', swmr=True)
test2 = hd.File('/mnt/nas2/results/IntermediateResults/Camelyon/ultrafast/test_data2/patches.hdf5', 'r', libver='latest', swmr=True)
pannuke= hd.File('/mnt/nas2/results/IntermediateResults/Camelyon/pannuke/patches_fix.hdf5', 'r', libver='latest', swmr=True)

global datasetss
datasetss={'cam16':cam16,'all500':all500,'extra17':extra17, 'tumor_extra17':tumor_extra17, 'test_data2': test2, 'pannuke':pannuke}

global concept_db
concept_db = hd.File('../../data/normalized_cmeasures/concept_values_def.h5py','r')
#concept_db = hd.File('/mnt/nas2/results/IntermediateResults/Mara/MICCAI2020/MELBA_only_contrast_n.hd','r')
#/mnt/nas2/results/IntermediateResults/Mara/MICCAI2020/MELBA_normalized_concepts.hd', 'r')
# Note: nuclei_concepts not supported yet
#global nuclei_concepts
#nuclei_concepts=hd.File('/mnt/nas2/results/IntermediateResults/Mara/MICCAI2020/normalized_nuclei_concepts_db_new_try_def.hdf5','r')


# Note: with uncertainty training we do not need CONCEPT ALPHAS (or LAMBDAs?) anymore
# We still need to specify the learning rate of the gradient reversal (if we want to change it) or of the additional task.
# I think we want to reduce it of a tenth or so

#SYSTEM CONFIGS 
CONFIG_FILE = '../../doc/config.cfg'
COLOR = True
#global new_folder
#new_folder = getFolderName()
#folder_name=EXPERIMENT_TYPE
#new_folder = 'results/'+folder_name #new_folder
#os.mkdir(new_folder)

# creating an ERR.log file to keep track of issues happened during model run
#global error_log
#error_log=open(new_folder+'/ERR.log', 'w')
#llg.basicConfig(filename=os.path.join(ne
BATCH_SIZE = 32

# SAVE FOLD
f=open(model_folder+"/seed.txt","r")
seed=1001#int(f.read())
if verbose:  print(seed)
#f.write(str(seed))
f.close()

# SET PROCESS TITLE
setproctitle.setproctitle('{}'.format(EXPERIMENT_TYPE))

# SET SEED
np.random.seed(seed)
tf.set_random_seed(seed)

# DATA SPLIT CSVs 
train_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/train_shuffle.csv', 'r') # How is the encoding of .csv files ?
val_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/val_shuffle.csv', 'r')
test_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/test_shuffle.csv', 'r')
train_list=train_csv.readlines()
val_list=val_csv.readlines()
test_list=test_csv.readlines()
test2_csv = open('/mnt/nas2/results/IntermediateResults/Camelyon/test2_shuffle.csv', 'r')
test2_list=test2_csv.readlines()
test2_csv.close()
train_csv.close()
val_csv.close()
test_csv.close()
data_csv=open('../../doc/pannuke_data_shuffle.csv', 'r')
data_list=data_csv.readlines()
data_csv.close()

1001


In [6]:
# STAIN NORMALIZATION
def get_normalizer(patch, save_folder='../../results/'):
    normalizer = ReinhardNormalizer()
    normalizer.fit(patch)
    np.save('{}/normalizer'.format(save_folder),normalizer)
    np.save('{}/normalizing_patch'.format(save_folder), patch)
    #print('Normalisers saved to disk.')
    return normalizer
def normalize_patch(patch, normalizer):
    return np.float64(normalizer.transform(np.uint8(patch)))

global normalizer
db_name, entry_path, patch_no = get_keys(data_list[0])
normalization_reference_patch = datasetss[db_name][entry_path][patch_no]
normalizer = get_normalizer(normalization_reference_patch, save_folder='../../results/')

"""
Batch generators: 
They load a patch list: a list of file names and paths. 
They use the list to create a batch of 32 samples. 
"""
# Retrieve Concept Measures
def get_concept_measure(db_name, entry_path, patch_no, measure_type=''):
    ### note: The measures in the file should have been scaled beforehand
    # to have zero mean and unit std
    
    path=db_name+'/'+entry_path+'/'+str(patch_no)+'/'+measure_type
    #import pdb; pdb.set_trace()
    #print path
    #print concept_db[path]
    #print concept_db[path][0]
    try:
        cm=concept_db[path][0]
        return cm
    except:
        print("[ERR]: {}, {}, {}, {} with path {}".format(db_name, entry_path, patch_no, measure_type, path))
        return 1.
    
class DataGenerator(keras.utils.Sequence):
    def __init__(self, patch_list, concept=CONCEPT, batch_size=32, shuffle=True, data_type=0):
        self.batch_size=batch_size
        self.patch_list=patch_list
        self.shuffle=shuffle
        self.concept = concept
        self.data_type=data_type
        print 'data type:', data_type
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.patch_list)/self.batch_size))
    def __getitem__(self, index):
        indexes=self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        patch_list_temp=[self.patch_list[k] for k in indexes]
        #x, y, cm = self.__data_generation(self, patch_list_temp)
        self.patch_list_temp=patch_list_temp
        return self.__data_generation(self), None
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.patch_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    def __data_generation(self, patch_list_temp):
        patch_list_temp=self.patch_list_temp
        batch_x=np.zeros((len(patch_list_temp), 224,224,3))
        batch_y=np.zeros(len(patch_list_temp))
        i=0
        for line in patch_list_temp:
            db_name, entry_path, patch_no = get_keys(line)
            patch=datasetss[db_name][entry_path][patch_no]
            patch=normalize_patch(patch, normalizer)
            patch=keras.applications.inception_v3.preprocess_input(patch) 
            label = get_class(line, entry_path) 
            if self.data_type!=0:
                label=get_test_label(entry_path)
            batch_x[i]=patch
            batch_y[i]=label
            i+=1
        #batch_x=np.asarray(batch_x, dtype=np.float32)
        #batch_y=np.asarray(batch_y, dtype=np.float32)
        #return [batch_x, batch_y, np.ones(len(patch_list_temp))], None
        generator_output=[batch_x, batch_y]#np.ones(len(patch_list_temp)
        for c in self.concept:
            batch_concept_values=np.zeros(len(patch_list_temp))
            i=0
            for line in patch_list_temp:
                db_name, entry_path, patch_no = get_keys(line)
                #print 'line: {}, cmeasure: '.format(line),get_concept_measure(db_name, entry_path, patch_no, measure_type=c)
                batch_concept_values[i]=get_concept_measure(db_name, entry_path, patch_no, measure_type=c)
                i+=1
            #print 'bcm: ', batch_concept_values
            generator_output.append(batch_concept_values)
        return generator_output

def get_batch_data(patch_list, batch_size=32):
    num_samples=len(patch_list)
    while True:
        offset = 0
        for offset in range(0,num_samples, batch_size):
            batch_x = []
            batch_y = []
            #concepts=['full_contrast','full_correlation','narea', 'ncount','nuclei_contrast','nuclei_correlation']
            #for c in concepts:
            #batch_concept_values[c]=[]
            batch_samples=patch_list[offset:offset+batch_size]
            for line in batch_samples[:(num_samples//batch_size)*batch_size]:
                db_name, entry_path, patch_no = get_keys(line)
                patch=datasetss[db_name][entry_path][patch_no]
                patch=normalize_patch(patch, normalizer)
                patch=keras.applications.inception_v3.preprocess_input(patch) 
                label = get_class(line, entry_path) 
                batch_x.append(patch)
                batch_y.append(label)
            batch_x=np.asarray(batch_x, dtype=np.float32)
            batch_y=np.asarray(batch_y, dtype=np.float32)
            generator_output=[batch_x, batch_y]
            
            for c in CONCEPT:
                batch_concept_values=[]
                for line in batch_samples[:(num_samples//batch_size)*batch_size]:
                    batch_concept_values.append(get_concept_measure(db_name, entry_path, patch_no, measure_type=c))
                batch_concept_values=np.asarray(batch_concept_values, dtype=np.float32)
                generator_output.append(batch_concept_values)
            #batch_domain=keras.utils.to_categorical(batch_domain, num_classes=6)
            yield generator_output, None
            
def get_test_batch(patch_list, batch_size=32):
    num_samples=len(patch_list)
    while True:     
        for offset in range(0,num_samples, batch_size):
            batch_x = []
            batch_y = []
            #batch_concept_values={}
            #concepts=['full_contrast','full_correlation','narea', 'ncount','nuclei_contrast','nuclei_correlation']
            #for c in concepts:
            #    batch_concept_values[c]=[]
            batch_samples=patch_list[offset:offset+batch_size]
            for line in batch_samples:
                db_name, entry_path, patch_no = get_keys(line)
                patch=datasetss[db_name][entry_path][patch_no]
                patch=normalize_patch(patch, normalizer)
                patch=keras.applications.inception_v3.preprocess_input(patch)
                #patch=keras.applications.resnet50.preprocess_input(patch)
                label = get_test_label(entry_path)
                batch_x.append(patch)
                batch_y.append(label)
            batch_x=np.asarray(batch_x, dtype=np.float32)
            batch_y=np.asarray(batch_y, dtype=np.float32)
            generator_output=[batch_x, batch_y]
            for c in CONCEPT:
                batch_concept_values=[]
                for line in batch_samples[:(num_samples//batch_size)*batch_size]:
                    batch_concept_values.append(get_concept_measure(db_name, entry_path, patch_no, measure_type=c))
                batch_concept_values=np.asarray(batch_concept_values, dtype=np.float32)
                generator_output.append(batch_concept_values)
            yield generator_output, None

Using brightness standardization


In [7]:
#import matplotlib as mpl   
#mpl.use('Agg')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger('tensorflow').disabled = True
from keras import *
import setproctitle
SERVER_NAME = 'ultrafast'
import time
import sys
import shutil
## Adding PROCESS_UC1 utilities
sys.path.append('../../lib/TASK_2_UC1/')
from models import *
from util import otsu_thresholding
from extract_xml import *
from functions import *                   
sys.path.append('../../lib/')
from mlta import *
import math
import keras.callbacks as callbacks
from keras.callbacks import Callback

In [8]:
"""         
Building guidable model 
"""
def get_baseline_model(hp_lambda=0., domain=False, c_list=[]):
    base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(224,224,3))
    layers_list=['conv2d_92', 'conv2d_93', 'conv2d_88', 'conv2d_89', 'conv2d_86']
    #layers_list=[]
    for i in range(len(base_model.layers[:])):
        layer=base_model.layers[i]
        if layer.name in layers_list:
            print layer.name
            layer.trainable=True
        else:
            layer.trainable = False
    feature_output=base_model.layers[-1].output
    gap_layer_output = keras.layers.GlobalAveragePooling2D()(feature_output)
    feature_output = Dense(2048, activation='relu', name='finetuned_features1',kernel_regularizer=keras.regularizers.l2(0.01))(gap_layer_output) 
    feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
    feature_output = Dense(512, activation='relu', name='finetuned_features2',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
    feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
    feature_output = Dense(256, activation='relu', name='finetuned_features3',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
    feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
    grl_layer=GradientReversal(hp_lambda=hp_lambda)
    feature_output_grl = grl_layer(feature_output)
    if domain:
        domain_adversarial = keras.layers.Dense(7, activation = keras.layers.Activation('softmax'), name='domain_adversarial')(feature_output_grl)
    finetuning = Dense(1,name='predictions')(feature_output)
    ## here you need to check how many other concepts you have apart from domain adversarial
    # then you add one layer per each. 
    if domain:
        output_nodes=[finetuning, domain_adversarial]
    else: 
        output_nodes=[finetuning]
    for c in c_list:
        if c!='domain':
            concept_layer=  keras.layers.Dense(1, activation = keras.layers.Activation('linear'), name='extra_{}'.format(c.strip(' ')))(feature_output)
            output_nodes.append(concept_layer)
    model = Model(input=base_model.input, output=output_nodes)
    model.grl_layer=grl_layer
    return model

In [9]:
base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(224,224,3))
layers_list=['conv2d_92', 'conv2d_93', 'conv2d_88', 'conv2d_89', 'conv2d_86']
#layers_list=[]
for i in range(len(base_model.layers[:])):
    layer=base_model.layers[i]
    if layer.name in layers_list:
        print layer.name
        layer.trainable=True
    else:
        layer.trainable = False
feature_output=base_model.layers[-1].output
gap_layer_output = keras.layers.GlobalAveragePooling2D()(feature_output)
feature_output = keras.layers.Dense(2048, activation='relu', name='finetuned_features1',kernel_regularizer=keras.regularizers.l2(0.01))(gap_layer_output) 
feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
feature_output = keras.layers.Dense(512, activation='relu', name='finetuned_features2',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
feature_output = keras.layers.Dense(256, activation='relu', name='finetuned_features3',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
finetuning = keras.layers.Dense(1,name='predictions')(feature_output)
regression_output = keras.layers.Dense(1, activation = keras.layers.Activation('linear'), name='concept_regressor')(gap_layer_output)
model = keras.Model(input=base_model.input, output=[finetuning, regression_output])
# Callbacks
def compute_mse(labels, predictions):
    errors = labels - predictions
    sum_squared_errors = np.sum(np.asarray([pow(errors[i],2) for i in range(len(errors))]))
    mse = sum_squared_errors / len(labels)
    return mse
class eval_model(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        f = open('{}/val_by_epoch.txt'.format(new_folder), 'a')
        pred_ = self.model.predict_generator(val_generator2, steps = len(val_list)// BATCH_SIZE, workers=4, use_multiprocessing=False)#// hvd.size()) 
        #import pdb; pdb.set_trace()
        y_true = pred_[:,0]
        y_pred = pred_[:,2]
        print 'y_true: ', y_true
        print 'y_pred: ', y_pred
        val_acc = my_accuracy_np(y_true, y_pred)
        cm_true = pred_[:, 1]
        cm_pred = pred_[:,3]
        print 'cm_true: ', cm_true
        print 'cm_pred: ', cm_pred
        
        val_r2 = r_square_np(cm_true, cm_pred)
        val_mse = compute_mse(cm_true, cm_pred)
        print("Val acc: {}, r2: {}, MSE: {}".format(val_acc, val_r2, val_mse))
        report_val_acc.append(val_acc)
        report_val_r2.append(val_r2)
        report_val_mse.append(val_mse)
        f.write("Val acc: {}, r2: {}, mse: {}".format(val_acc, val_r2, val_mse))
        
        train_pred_ = self.model.predict_generator(train_generator2, steps=100, workers=1, use_multiprocessing=False)
        cm_true = train_pred_[:, 1]
        cm_pred = train_pred_[:,3]
        print 'train_cmtrue: ', cm_true
        print 'train_cmpred: ', cm_pred
        train_r2 = r_square_np(cm_true, cm_pred)
        train_mse = compute_mse(cm_true, cm_pred)
        print("Train r2: {}, MSE: {}".format(train_r2, train_mse))

conv2d_88
conv2d_89
conv2d_92
conv2d_93
conv2d_86


In [10]:
# END Callbacks
#
def keras_mse(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.mean_squared_error(y_true, y_pred))
    #return tf.keras.losses.mean_squared_error(y_true, y_pred)

def bbce(y_true, y_pred):
    # we use zero weights to set the loss to zero for unlabeled data
    verbose=0
    zero= tf.constant(-1, dtype=tf.float32)
    where = tf.not_equal(y_true, zero)
    where = tf.reshape(where, [-1])
    indices=tf.where(where) #indices where the item of y_true is NOT -1
    indices = tf.reshape(indices, [-1])
    sliced_y_true = tf.nn.embedding_lookup(y_true, indices)
    sliced_y_pred = tf.nn.embedding_lookup(y_pred, indices)
    n1 = tf.shape(indices)[0] #number of train images in batch
    batch_size = tf.shape(y_true)[0]
    n2 = batch_size - n1 #number of test images in batch
    sliced_y_true = tf.reshape(sliced_y_true, [n1, -1])
    n1_ = tf.cast(n1, tf.float32)
    n2_ = tf.cast(n2, tf.float32)
    multiplier = (n1_+ n2_) / n1_
    zero_class = tf.constant(0, dtype=tf.float32)
    where_class_is_zero=tf.cast(tf.reduce_sum(tf.cast(tf.equal(sliced_y_true, zero_class), dtype=tf.float32)), dtype=tf.float32)
    if verbose:
        where_class_is_zero=tf.Print(where_class_is_zero,[where_class_is_zero],'where_class_is_zero: ')
    class_weight_zero = tf.cast(tf.divide(n1_, 2. * tf.cast(where_class_is_zero, dtype=tf.float32)+0.001), dtype=tf.float32)
    
    if verbose:
        class_weight_zero=tf.Print(class_weight_zero,[class_weight_zero],'class_weight_zero: ')
    one_class = tf.constant(1, dtype=tf.float32)
    where_class_is_one=tf.cast(tf.reduce_sum(tf.cast(tf.equal(sliced_y_true, one_class), dtype=tf.float32)), dtype=tf.float32)
    if verbose:
        where_class_is_one=tf.Print(where_class_is_one,[where_class_is_one],'where_class_is_one: ')
        n1_=tf.Print(n1_,[n1_],'n1_: ')
    class_weight_one = tf.cast(tf.divide(n1_, 2. * tf.cast(where_class_is_one,dtype=tf.float32)+0.001), dtype=tf.float32)
    class_weight_zero =  tf.constant(23477.0/(23477.0+123820.0), dtype=tf.float32)
    class_weight_one =  tf.constant(123820.0/(23477.0+123820.0), dtype=tf.float32)
    A = tf.ones(tf.shape(sliced_y_true), dtype=tf.float32) - sliced_y_true 
    A = tf.scalar_mul(class_weight_zero, A)
    B = tf.scalar_mul(class_weight_one, sliced_y_true) 
    class_weight_vector=A+B
    ce = tf.nn.sigmoid_cross_entropy_with_logits(labels=sliced_y_true,logits=sliced_y_pred)
    ce = tf.multiply(class_weight_vector,ce)
    return tf.reduce_mean(ce)

# Custom loss layer
from keras.initializers import Constant
class CustomMultiLossLayer(Layer):
    def __init__(self, nb_outputs=2, **kwargs):
        self.nb_outputs = nb_outputs
        self.is_placeholder = True
        super(CustomMultiLossLayer, self).__init__(**kwargs)
        
    def build(self, input_shape=None):
        self.log_vars = []
        for i in range(self.nb_outputs):
            self.log_vars += [self.add_weight(name='log_var' + str(i), shape=(1,), initializer=Constant(0.), trainable=True)]
        super(CustomMultiLossLayer, self).build(input_shape)
    
    def multi_loss(self,  ys_true, ys_pred):
        #print len(ys_true)
        assert len(ys_true) == self.nb_outputs and len(ys_pred) == self.nb_outputs
        loss = 0
        i=0
        for y_true, y_pred, log_var in zip(ys_true, ys_pred, self.log_vars):
            precision =K.exp(-log_var[0]) ###MODIFICATION HERE
            if i==0:
                pred_loss = bbce(y_true, y_pred)
                term = precision*pred_loss + 0.5 * log_var[0]  
                #term=tf.Print(term, [term], 'bbce: ')
            else:
                pred_loss = keras_mse(y_true, y_pred)
                #pred_loss=tf.Print(pred_loss, [pred_loss], 'MSE: ')
                term = 0.5 * precision * pred_loss + 0.5 * log_var[0]
                #term=tf.Print(term, [term], 'MSE: ')
            loss+=term
            term = 0.
            i+=1
        return K.mean(loss)
    
    def call(self, inputs):
        ys_true = inputs[:self.nb_outputs]
        ys_pred = inputs[self.nb_outputs:]
        loss = self.multi_loss(ys_true, ys_pred)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return K.concatenate(inputs, -1)
def get_trainable_model(baseline_model):
    inp = keras.layers.Input(shape=(224,224,3,), name='inp')
    y1_pred, y2_pred = baseline_model(inp)
    y1_true=keras.layers.Input(shape=(1,),name='y1_true')
    y2_true=keras.layers.Input(shape=(1,),name='y2_true')
    out = CustomMultiLossLayer(nb_outputs=2)([y1_true, y2_true, y1_pred, y2_pred])
    return Model(input=[inp, y1_true, y2_true], output=out)

""" Get trainable model with Hepistemic Uncertainty Weighted Loss """
t_m = get_trainable_model(model)

In [11]:
# getting baseline model 
baseline_ref = get_baseline_model(hp_lambda=0.)

In [12]:
baseline_ref.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 111, 111, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_96 (Activation)      (None, 111, 111, 32) 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

In [13]:
def grad_cam(input_model, image, layer_name,H=224,W=224):
    cls = np.argmax(input_model.predict(image))
    def normalize(x):
        """Utility function to normalize a tensor by its L2 norm"""
        return (x + 1e-10) / (K.sqrt(K.mean(K.square(x))) + 1e-10)
    """GradCAM method for visualizing input saliency."""
    y_c = input_model.get_layer('predictions').output#[0, cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]
    grads = normalize(grads)
    gradient_function = K.function([input_model.get_layer('input_1').input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)
    #print (cam)

    cam = np.maximum(cam, 0) # equivalent to RELU pass
    cam = cv2.resize(cam, (H, W))
    #cam = resize(cam, (H, W))
    # cam = zoom(cam,H/cam.shape[0])
    #cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam

In [14]:
def grad_cam_baseline(input_model, image, layer_name,H=224,W=224):
    cls = np.argmax(input_model.predict(image))
    def normalize(x):
        """Utility function to normalize a tensor by its L2 norm"""
        return (x + 1e-10) / (K.sqrt(K.mean(K.square(x))) + 1e-10)
    """GradCAM method for visualizing input saliency."""
    y_c = input_model.get_layer('predictions').output#[0, cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]
    grads = normalize(grads)
    gradient_function = K.function([input_model.get_layer('input_2').input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)
    #print (cam)

    cam = np.maximum(cam, 0) # equivalent to RELU pass
    cam = cv2.resize(cam, (H, W))
    #cam = resize(cam, (H, W))
    # cam = zoom(cam,H/cam.shape[0])
    #cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam

In [15]:
def grad_cam_concept(input_model, image, layer_name,H=224,W=224):
    cls = np.argmax(input_model.predict(image))
    def normalize(x):
        """Utility function to normalize a tensor by its L2 norm"""
        return (x + 1e-10) / (K.sqrt(K.mean(K.square(x))) + 1e-10)
    """GradCAM method for visualizing input saliency."""
    y_c = input_model.get_layer('concept_regressor').output#[0, cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]
    grads = normalize(grads)
    gradient_function = K.function([input_model.get_layer('input_1').input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)
    #print (cam)

    cam = np.maximum(cam, 0) # equivalent to RELU pass
    cam = cv2.resize(cam, (H, W))
    #cam = resize(cam, (H, W))
    # cam = zoom(cam,H/cam.shape[0])
    #cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam

In [16]:
init=tf.global_variables_initializer() #initialize_all_variables()
sess=tf.Session()
sess.run(init)

In [17]:
t_m = get_trainable_model(model)
t_m.load_weights(model_folder+'/best_model.h5')x

In [18]:
test_generator_=get_test_batch(test_list, batch_size=32)

In [19]:
test_generator_=get_test_batch(test_list, batch_size=32)
[x_b, y_b, cm_b], _ = test_generator_.next()

[ERR]: all500, normal/level7/centre3/patient068/node1/patches, 148, domain with path all500/normal/level7/centre3/patient068/node1/patches/148/domain
[ERR]: all500, normal/level7/centre3/patient068/node1/patches, 148, domain with path all500/normal/level7/centre3/patient068/node1/patches/148/domain
[ERR]: all500, normal/level7/centre3/patient068/node1/patches, 148, domain with path all500/normal/level7/centre3/patient068/node1/patches/148/domain
[ERR]: all500, normal/level7/centre3/patient068/node1/patches, 148, domain with path all500/normal/level7/centre3/patient068/node1/patches/148/domain
[ERR]: all500, normal/level7/centre3/patient068/node1/patches, 148, domain with path all500/normal/level7/centre3/patient068/node1/patches/148/domain
[ERR]: all500, normal/level7/centre3/patient068/node1/patches, 148, domain with path all500/normal/level7/centre3/patient068/node1/patches/148/domain
[ERR]: all500, normal/level7/centre3/patient068/node1/patches, 148, domain with path all500/normal/l

ValueError: too many values to unpack

In [ ]:
baseline_ref.load_weights('../../results/BASEL/best_model.h5')

In [ ]:
heatmap=grad_cam(model, np.expand_dims(x_b[0], 0), 'mixed10')
heatmap_ref = grad_cam_baseline(baseline_ref, np.expand_dims(x_b[0],0), 'mixed10')

In [ ]:
BATCH_SIZE = 32
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
def evaluate_model(d_list, model, batch_size=BATCH_SIZE, test_type=''):
    batch_size=32
    #d_list=test_list[:100]
    #t_gen=DataGenerator(d_list, concept=CONCEPT, batch_size=BATCH_SIZE, data_type=0)
    test_generator_=get_test_batch(d_list, batch_size=batch_size)
    steps=len(d_list)//batch_size
    print steps
    initial_lr = 1e-4
    opt = keras.optimizers.SGD(lr=initial_lr, momentum=0.9, nesterov=True)
    compile_model(t_m,opt,loss=None,metrics=None)
    callbacks = []
    y_true=np.zeros(len(d_list))
    y_pred=np.zeros((len(d_list),1))
    N=0
    all_cm=np.zeros(len(d_list))
    all_p_cm=np.zeros(len(d_list))

    while N<len(d_list):
        #x_b, y_b, cm_b = t_gen.__getitem__(N)
        [x_b, y_b, cm_b], _ = test_generator_.next()
        pred_ = t_m.predict([x_b, y_b, cm_b])
        y_p_b = pred_[:,2]
        cm_p_b = pred_[:,3]

        y_true[N:N+len(y_b)]=y_b.reshape(len(y_b))
        y_pred[N:N+len(y_p_b)]=y_p_b.reshape(len(y_p_b),1)
        all_p_cm[N:N+len(cm_p_b)]=cm_p_b.reshape(len(cm_p_b))
        all_cm[N:N+len(cm_b)]=cm_b
        N+=len(y_p_b)
    y_true=y_true.reshape((len(d_list),1))
    acc = my_accuracy(y_true, y_pred).eval(session=tf.Session())
    sliced_y_pred = tf.sigmoid(y_pred)
    y_pred_rounded = K.round(sliced_y_pred)
    acc_sc = accuracy_score(y_pred_rounded.eval(session=tf.Session()), y_true)
    print('accuracy: ', acc_sc)
    
    y_pred = sliced_y_pred.eval(session=tf.Session())
    #sliced_y_pred = tf.sigmoid(y_pred)
    #y_pred_rounded = K.round(sliced_y_pred)
    auc_score=sklearn.metrics.roc_auc_score(y_true,sliced_y_pred.eval(session=tf.Session()))
    
    '''   fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(1):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc_score
    plt.figure()
    lw = 2
    plt.plot(fpr[0], tpr[0], color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example AUC = {}'.format(roc_auc[0]))
    plt.legend(loc="lower right")
    plt.show()'''    
    print 'auc: {}'.format(auc_score)
    #auc_record = open('{}/auc_{}.txt'.format(model_folder,test_type), 'w')
    #auc_record.write('{}'.format(roc_auc[0]))
    #auc_record.close()
    return all_cm, all_p_cm, auc_score
all_cm_t, all_p_cm_t, roc_auc=evaluate_model(test_list[:],t_m, test_type='internal')

In [ ]:
plt.rcParams['figure.figsize']=(13,17)
for i in range(1, len(x_b)):
    print i
    heatmap=grad_cam(model, np.expand_dims(x_b[i], 0), 'mixed10')
    heatmap_concept=grad_cam_concept(model, np.expand_dims(x_b[i], 0), 'mixed10')
    heatmap_ref = grad_cam_baseline(baseline_ref, np.expand_dims(x_b[i],0), 'mixed10')
    plt.figure()
    plt.subplot(1,4,1)
    plt.imshow(x_b[i])
    plt.title('orig')
    plt.axis('off')
    plt.subplot(1,4,2)
    plt.imshow(x_b[i])
    plt.title('guided MT')
    plt.axis('off')
    plt.imshow(heatmap, alpha=0.7)
    plt.subplot(1,4,3)
    plt.imshow(x_b[i])
    plt.imshow(heatmap_concept, alpha=0.7)
    plt.title('{}'.format(CONCEPT))
    plt.axis('off')
    plt.subplot(1,4,4)
    plt.imshow(x_b[i])
    plt.imshow(heatmap_ref, alpha=0.7)
    plt.title('baseline MT')
    plt.axis('off')
    plt.savefig('./guided/{}_{}'.format(CONCEPT,i))
    